In [1]:
# 安装命令：pip install -U libsvm-official --trusted-host pypi.tuna.tsinghua.edu.cn
from libsvm.svmutil import *
import numpy as np
from sklearn.metrics import roc_curve, auc

In [2]:
def data_read_mat(file_name):   
    num_list = []
    
    with open(file_name,"r",encoding='UTF-8') as file:
        for l in file:
            l = l.split(',')  # l为列表类型
            
            list_k = []
            for j in range(3):
                # [a,1,b,3,c,2,draw]为例，经过下面代码将会变为[0, 1, 1, 3, 2, 2]
                '''ord()函数用于将字符转换为整数'''
                list_k.append(ord(l[j*2]) - ord('a'))   # 0、2、4位是字母。将字母a化为数字0，以此类推
                list_k.append(ord(l[j*2 + 1]) - ord('0'))  # 数字位还是保留原来的数字
                
            if(l[6][0] == 'd'):
                list_k.append(1)   # draw为正样本
            else:
                list_k.append(-1)    # 其余情况都为负样本
                
            num_list.append(list_k)  # 此时list_k中第一条为[a,1,b,3,c,2,draw] ——> [0, 1, 1, 3, 2, 2, 1]
            
    num_mat = np.array(num_list,dtype="float")
    '''
    在此处是以numpy的二维数据矩阵的形式存储的，本以为使用numpy的数据进行运算可以使得
    训练的速度快一些，结果发现如果要往libsvm中的函数传入参数只能传入list型，不能传入numpy
    的数据类型。所以，后面又把数据类型转回了list型
    '''
    return num_mat   # 返回一个n*7的矩阵，前6列是三个坐标，第七列是标签

In [3]:
def data_deal(mat, len_train, len1, len_test, len2):
    '''
    mat: 大矩阵，其中包括训练数据和测试数据
    len_train:训练数据   5000
    len1: 输入坐标  6
    len_test: 测试数据
    len2: 标签  1
    '''
    np.random.shuffle(mat)  # 先将矩阵按行打乱
    # 然后根据要求对矩阵进行分割，第一部分是训练集，第二部分是测试集
    x_part1 = mat[0:len_train, 0:len1]   # 训练数据的坐标，shape为(5000,6)
    x_part2 = mat[len_train:, 0:len1]    # 测试数据的坐标
    y_part1 = mat[0:len_train, len1]     # 训练数据的标签
    y_part2 = mat[len_train:, len1]      # 测试数据的标签
    
    # 样本归一化：在训练样本上求出每个维度的均值和标准差，在训练和测试样本上同时归一化
    avgX = np.mean(x_part1, axis=0)    # axis=0，计算每一列的均值
    stdX = np.std(x_part1, axis=0)
    
    # 将训练集和测试集都进行归一化处理
    for data in x_part1:   # 训练集 (5000，6)
        for j in range(len(data)):   # j为0~5
            data[j] = (data[j] - avgX[j]) / stdX[j]
    for data in x_part2:   # 测试集
        for j in range(len(data)):
            data[j] = (data[j] - avgX[j]) / stdX[j]

    return x_part1, y_part1, x_part2, y_part2   # 返回的依次是训练数据，训练数据的标签，测试数据，测试数据的标签

In [4]:
def TrainModel(CScale, gammaScale, prob):
    '''
    CScale: 参数C的取值序列
    gammaScale: 参数γ的取值序列
    prob: 训练集合对应的标签
    '''
    maxACC = 0             # 最高正确率
    maxACC_C = 0           # 最优参数C
    maxACC_gamma = 0       # 最优参数γ
 
    # 嵌套循环
    for C in CScale:
        C_ = pow(2, C)  # 2的C次方
        for gamma in gammaScale:
            gamma_ = pow(2, gamma)  # 2的gamma次方
            
            # 设置训练的参数，其中-q表示静默模式，不输出训练过程信息
            param = svm_parameter('-t 2 -c ' + str(C_) + ' -g ' + str(gamma_) + ' -v 5 -q')
            ACC = svm_train(prob, param)  # 进行训练，但是传回的不是训练模型而是5折交叉验证的准确率
            
            # 更新数据
            if (ACC > maxACC):
                maxACC = ACC
                maxACC_C = C
                maxACC_gamma = gamma
                
    return maxACC, maxACC_C, maxACC_gamma  

In [5]:
def getNewList(L, U, step):   
    l = []
    while(L < U):
        l.append(L)
        L += step
    return l

In [6]:
def TrainModelSVM(data, label, iter, model_file):
    '''
    data: 数据
    label: 标签
    iter: 训练次数，在原先的MATLAB代码中的次数是两次
    model_file: 模型的保存位置
    '''
    
    # 将数据转换成list型的数据。因为在svm的函数中好像只能传入list型的数据进行训练使用
    X = data.tolist()
    Y = label.tolist()
    
    CScale = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]     # 参数C的取值为2^C
    gammaScale = [-15, -13, -11, -9, -7, -5, -3, -1, 1, 3]   # 参数γ的取值为2^γ
    cnt = iter
    step = 2  # 用于重新生成CScale和gammaScale序列
    maxACC = 0         # 训练过程中的最大正确率
    bestACC_C = 0      # 训练过程中的最优参数C
    bestACC_gamma = 0  # 训练过程中的最优参数γ
    prob = svm_problem(Y, X)  # 传入数据
    
    while(cnt):
        # 用传入的参数序列进行训练，返回的是此次训练的最高正确率、最优参数C、最优参数γ
        maxACC_train, maxACC_C_train, maxACC_gamma_train = TrainModel(CScale, gammaScale, prob)   # prob为训练集合对应的标签
        # 数据更新
        if(maxACC_train > maxACC):
            maxACC = maxACC_train
            bestACC_C = maxACC_C_train
            bestACC_gamma = maxACC_gamma_train
            
        # 根据返回的参数重新生成CScale序列和gammaScale序列用于再次训练，下一次训练的C参数和γ参数的精度会比之前更高
        # step就是CScale序列和gammaScale序列相邻两个数之间的间隔
        new_step = step*2/10
        CScale = getNewList(maxACC_C_train - step, maxACC_C_train + step + new_step, new_step)
        gammaScale = getNewList(maxACC_gamma_train - step, maxACC_gamma_train + step + new_step, new_step)
        cnt -= 1
        
    # 获得最优参数后计算出对应的C和γ，并且训练获得最优模型
    C = pow(2, bestACC_C)
    gamma = pow(2, bestACC_gamma)
    param = svm_parameter('-t 2 -c ' + str(C) + ' -g ' + str(gamma))
    model = svm_train(prob, param)    # 准确率
    svm_save_model(model_file, model) # 保存模型
    return model

In [7]:
def main():
    data_file = r"D:\Jupyter Notebook\SVM\krkopt.data"  # 数据存放的位置
    mode_file = r"D:\Jupyter Notebook\SVM\model_file"   # 训练模型保存的位置
    data_mat = data_read_mat(data_file)  # 从文件中读取数据并处理
    
    # 以下是对训练数据进行分配
    train = 5000   # 5000组数据作为训练数据
    test  = len(data_mat) - 5000  # 剩下的数据（23056组）作为测试数据
    x_len = 6   # 输入数据的维度是6维，即三个棋子的坐标
    y_len = len(data_mat[0]) - x_len  # 输出的数据是1维，即两种结果
    iter = 2   # 训练的次数，训练的次数越多，参数就调整的精度越高
    x_train, y_train, x_test, y_test = data_deal(data_mat, train, x_len, test, y_len)   # 对数据进行分割
    
    if (input("是否需要进行训练？") == 'y'):  # 如果输入y就会进行训练，否则就可以直接使用之前训练完成的模型
        model = TrainModelSVM(x_train, y_train, iter, mode_file)  # 传入输入数据、标签进行模型的训练
    else:
        model = svm_load_model(mode_file)  # 直接加载现有模型
        
    X = x_test.tolist()  # 将测试集的坐标转换成list
    Y = y_test.tolist()  # 将测试集的标签转换成list
    # print(Y[:])
    p_labs, p_acc, p_vals = svm_predict(Y, X, model)
    
    fpr, tpr, threshold = roc_curve(y_test, p_labs)  # 计算真正率tpr和假正率fpr
    roc_auc = auc(fpr, tpr)  # 计算auc的值，auc就是曲线包围的面积，越大越好
    print(roc_auc)

In [8]:
if __name__ == "__main__":
    main()

是否需要进行训练？y
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 90.08%
Cross Validation Accuracy = 91.94%
Cross Val

### 博客：https://zhuanlan.zhihu.com/p/537715035